<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/rain_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **S4E10 - Rain Forecast**

The table below shows the column names, their value formats, and their description.

Index| Features      |Format             |Description
-----|---------------|-------------------|-----------------------
1    |Date Time      |01.01.2009 00:10:00|Date-time reference
2    |p (mbar)       |996.52             |The pascal SI derived unit of pressure used to quantify internal pressure. Meteorological reports typically state atmospheric pressure in millibars.
3    |T (degC)       |-8.02              |Temperature in Celsius
4    |Tpot (K)       |265.4              |Temperature in Kelvin
5    |Tdew (degC)    |-8.9               |Temperature in Celsius relative to humidity. Dew Point is a measure of the absolute amount of water in the air, the DP is the temperature at which the air cannot hold all the moisture in it and water condenses.
6    |rh (%)         |93.3               |Relative Humidity is a measure of how saturated the air is with water vapor, the %RH determines the amount of water contained within collection objects.
7    |VPmax (mbar)   |3.33               |Saturation vapor pressure
8    |VPact (mbar)   |3.11               |Vapor pressure
9    |VPdef (mbar)   |0.22               |Vapor pressure deficit
10   |sh (g/kg)      |1.94               |Specific humidity
11   |H2OC (mmol/mol)|3.12               |Water vapor concentration
12   |rho (g/m ** 3) |1307.75            |Airtight
13   |wv (m/s)       |1.03               |Wind speed
14   |max. wv (m/s)  |1.75               |Maximum wind speed
15   |wd (deg)       |152.3              |Wind direction in degrees

## Intro: Problem Statement

<div class="container">
        <p class="mb-2">
            Analyze weather data over a 5-day period to identify patterns and correlations between various meteorological factors such as pressure, temperature, humidity, and rainfall. This can help in understanding how these variables influence daily weather conditions.
        </p>
        <h1 class="mb-2">Dataset Description</h1>
        <p class="mb-2">The dataset captures weather data over five days with the following features:</p>
        <ul class="mb-2">
            <li class="mb-2"><strong>day:</strong> Sequential day number.</li>
            <li class="mb-2"><strong>pressure (hPa):</strong> Atmospheric pressure measured in hectopascals.</li>
            <li class="mb-2"><strong>maxtemp (°C):</strong> Maximum temperature recorded on the day.</li>
            <li class="mb-2"><strong>temperature (°C):</strong> Average temperature of the day.</li>
            <li class="mb-2"><strong>mintemp (°C):</strong> Minimum temperature recorded on the day.</li>
            <li class="mb-2"><strong>dewpoint (°C):</strong> Temperature at which air becomes saturated with moisture.</li>
            <li class="mb-2"><strong>humidity (%):</strong> Relative humidity percentage.</li>
            <li class="mb-2"><strong>cloud (%):</strong> Cloud cover percentage.</li>
            <li class="mb-2"><strong>rainfall:</strong> Indicates if rainfall occurred ("yes"/"no").</li>
            <li class="mb-2"><strong>sunshine (hours):</strong> Total sunshine hours.</li>
            <li class="mb-2"><strong>winddirection (°):</strong> Direction of the wind in degrees.</li>
            <li class="mb-2"><strong>windspeed (km/h):</strong> Speed of the wind.</li>
        </ul>
        <p class="mb-2">
            This dataset is ideal for basic weather pattern analysis and visualization.
        </p>
    </div>

## 1.0 Libraries

In [1]:
%%capture
# #!pip install -qq pytorch_tabnet
!pip install optuna
# !pip install catboost
# #!pip install --upgrade numpy
# #!pip install optuna-integration-pytorch-tabnet

# #from pytorch_tabnet.tab_model import TabNetRegressor

!pip install --upgrade category-encoders
!pip install optuna-integration
!pip install colorama
# #!pip install pyfiglet
!pip install keras-tuner --upgrade
# !pip install keras-nlp
# !pip install BorutaShap
# !pip install --upgrade scikit-learn
# !pip install scikit-lego
!pip install skops

# #from pytorch_tabnet.tab_model import TabNetRegressor

In [2]:
# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
import skops.io as sio
#import calplot as cal

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Set Style
sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5});
sns.despine(left=True, bottom=True, top=False, right=False);
mpl.rcParams['figure.dpi'] = 120;
mpl.rc('axes', labelsize=12);
plt.rc('xtick',labelsize=10);
plt.rc('ytick',labelsize=10);

mpl.rcParams['axes.spines.top'] = False;
mpl.rcParams['axes.spines.right'] = False;
mpl.rcParams['axes.spines.left'] = True;

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)

# tool for Excel:
from openpyxl import load_workbook, Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.worksheet.cell_range import CellRange

from openpyxl.formatting import Rule
from openpyxl.styles import Font, PatternFill, Border
from openpyxl.styles.differential import DifferentialStyle

#from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

from keras.utils import FeatureSpace, plot_model

# Import libraries for Hypertuning
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, GridSearch, BayesianOptimization

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
#import skops.io as sio
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             root_mean_squared_log_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)
# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              StackingClassifier,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
from openpyxl import load_workbook

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.count import CountEncoder
from category_encoders import TargetEncoder

warnings.filterwarnings('ignore')
#import pyfiglet
#plt.style.use('fivethirtyeight')

<Figure size 640x480 with 0 Axes>

In [3]:
sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
#         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# import font colors
from colorama import Fore, Style, init

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

init(autoreset=True)

<Figure size 960x660 with 0 Axes>

In [4]:
from tqdm import tqdm
from itertools import product

import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
import torch

import warnings
warnings.filterwarnings("ignore")

# Connect to Colab:#
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.0 Loading and Preprocessing Data


In [5]:
df_subm = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/sample_submission.csv", index_col=0)

validation = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/X_enc_y_old_all_ext.csv")
train = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/X_enc_y_new_all_ext.csv")
test = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/test_enc_all_ext.csv")

In [6]:
#df_train_old.isna().sum()

In [7]:
train.shape, test.shape, validation.shape

((2190, 31), (730, 30), (365, 31))

In [8]:
df_all_new = pd.concat([train, test], axis=0)
print(df_all_new.shape)
df_all_new.head(3)

(2920, 31)


,pressure,temparature,dewpoint,humidity,cloud,sunshine,windspeed,Wdsx,Wdsy,Year_sin,Year_cos,sem_sin,sem_cos,m4_sin,m4_cos,m6_sin,m6_cos,m8_sin,m8_cos,temp_diff,dewpoint_spread,range_dist_cloud,cloud_Humidity_Interaction,cloud_sunshine,rolling_cloud_mean,rolling_humidity_mean,cloud_ratio,humidity_ratio,mean,std,rainfall
0,0.654955,-0.629296,-0.183091,0.638922,0.692700,-0.745242,-0.460377,0.266192,0.404027,0.017213,0.999852,0.034422,0.999407,0.051620,0.998667,0.103102,0.994671,0.137279,0.990532,-1.885535,-1.201297,-1.147765,0.202762,-0.667113,1.179413,1.133104,-0.012575,-0.002483,-0.663821,-0.432092,1.0
1,1.019030,-1.329863,-0.924418,1.616306,0.853347,-1.044309,0.013515,0.627866,0.523115,0.034422,0.999407,0.068802,0.997630,0.103102,0.994671,0.205104,0.978740,0.271958,0.962309,-2.468657,-1.047000,-1.714207,1.785597,-1.143049,1.316221,2.001867,0.056245,0.489094,-0.509969,1.260600,1.0
2,1.816533,-1.481337,-2.054941,-0.827153,-1.502805,1.212287,-0.369632,0.107086,0.537809,0.051620,0.998667,0.103102,0.994671,0.154309,0.988023,0.304921,0.952378,0.401488,0.915864,0.382156,1.678918,-1.102818,-1.006968,0.774952,0.024150,0.843517,-1.556966,-1.394965,-1.414627,0.788521,1.0


In [9]:
df_all_new.tail(3)

,pressure,temparature,dewpoint,humidity,cloud,sunshine,windspeed,Wdsx,Wdsy,Year_sin,Year_cos,sem_sin,sem_cos,m4_sin,m4_cos,m6_sin,m6_cos,m8_sin,m8_cos,temp_diff,dewpoint_spread,range_dist_cloud,cloud_Humidity_Interaction,cloud_sunshine,rolling_cloud_mean,rolling_humidity_mean,cloud_ratio,humidity_ratio,mean,std,rainfall
727,1.573820,-1.026915,-0.905885,1.249787,1.121091,-1.044309,0.013515,0.627866,0.523115,-3.442161e-02,0.999407,-6.880243e-02,0.997630,-1.031017e-01,0.994671,-2.051045e-01,0.978740,-2.719582e-01,0.962309,-0.136173,-0.275514,1.099351,1.826335,-1.143049,1.361823,1.288241,0.261176,0.543936,1.075949,0.195290,NaN
728,0.134842,-0.742902,-0.108959,1.494133,0.960444,-1.044309,1.788088,1.374918,1.376816,-1.721336e-02,0.999852,-3.442161e-02,0.999407,-5.161967e-02,0.998667,-1.031017e-01,0.994671,-1.372788e-01,0.990532,-1.885537,-1.715621,-1.049108,1.789645,-1.143049,1.387882,1.877758,0.110873,0.439198,0.778698,1.009724,NaN
729,1.261754,-0.970113,-1.313614,-0.338461,0.746249,-0.989933,2.907279,1.368967,2.235567,-2.449294e-16,1.000000,-4.898587e-16,1.000000,-7.347881e-16,1.000000,-1.469576e-15,1.000000,-1.959435e-15,1.000000,0.641322,1.010296,1.462053,1.115856,-1.055532,1.453028,1.691595,-0.102823,-1.322370,1.730059,1.154792,NaN


In [10]:
step = 1

past = 7
future = 0
learning_rate = 0.01
batch_size = 64
epochs = 21

test_final = df_all_new.iloc[2190-past+1:,:]

In [11]:
# Train data
x_train = train.iloc[:,:-1].values
y_train = train.iloc[past-1:,-1].values.reshape(-1,1)
print(x_train.shape, y_train.shape)
# Valid Data
x_valid = validation.iloc[:,:-1].values
y_valid = validation.iloc[past-1:,-1].values.reshape(-1,1)
print(x_valid.shape, y_valid.shape)
# Test Data
x_test = test_final.iloc[:,:-1].values
y_test = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test.shape, y_test.shape)

(2190, 30) (2184, 1)
(365, 30) (359, 1)
(736, 30) (730, 1)


## 3.0 Dataset Management Functions

## Version 0

####  **3.1 Train Dataset**

In [12]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (2184, 7, 30)
Target shape: (2184, 1)


In [13]:
# Find indices of 0s
indices_of_zeros = np.where(targets.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones = np.where(targets.numpy() == 1)[0]

In [14]:
indices_of_zeros.shape,indices_of_ones.shape

((539,), (1645,))

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [15]:
pos_features = inputs.numpy()[indices_of_ones]
neg_features = inputs.numpy()[indices_of_zeros]

pos_labels = targets.numpy()[indices_of_ones]
neg_labels = targets.numpy()[indices_of_zeros]

pos_features.shape,pos_labels.shape,neg_features.shape,neg_labels.shape

((1645, 7, 30), (1645, 1), (539, 7, 30), (539, 1))

In [16]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

In [17]:
for num, (features, label) in enumerate(pos_ds.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy())

Label sample 0:  [1]
Label sample 1:  [1]
Label sample 2:  [1]
Label sample 3:  [1]
Label sample 4:  [1]
Label sample 5:  [1]
Label sample 6:  [1]
Label sample 7:  [1]
Label sample 8:  [1]
Label sample 9:  [1]


In [18]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(64).prefetch(2)

In [19]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

0.625


#### **3.2 Validation Dataset**

In [20]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


#### **3.3 Test Dataset**

In [21]:
dataset_test = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


## Version 1

In [22]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10_000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs_val, targets_val = batch

print("Input shape:", inputs_val.numpy().shape)
print("Target shape:", targets_val.numpy().shape)

Input shape: (359, 7, 30)
Target shape: (359, 1)


In [23]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train.take(1):
    inputs_tr, targets_tr = batch

print("Input shape:", inputs_tr.numpy().shape)
print("Target shape:", targets_tr.numpy().shape)

Input shape: (2184, 7, 30)
Target shape: (2184, 1)


In [24]:
# Merge the arrays along the first axis
all_train = np.concatenate((inputs_val, inputs_tr), axis=0)
all_target = np.concatenate((targets_val, targets_tr), axis=0)
all_train.shape, all_target.shape

((2543, 7, 30), (2543, 1))

In [25]:
all_train.shape[0]-365, all_train.shape

(2178, (2543, 7, 30))

In [26]:
# Train data
x_train_v1 = all_train[:2178,:,:]
y_train_v1 = all_target[:2178,:]
print(x_train_v1.shape, y_train_v1.shape)
# Valid Data
x_valid_v1 = all_train[2178:,:,:]
y_valid_v1 = all_target[2178:,:]
print(x_valid_v1.shape, y_valid_v1.shape)
# Test Data
x_test_v1 = test_final.iloc[:,:-1].values
y_test_v1 = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test_v1.shape, y_test_v1.shape)

(2178, 7, 30) (2178, 1)
(365, 7, 30) (365, 1)
(736, 30) (730, 1)


####  **3.1 Train Dataset**

In [27]:
dataset_train_v1 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train_v1,
                                                                    y_train_v1,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (2172, 7, 7, 30)
Target shape: (2172, 1)


In [28]:
# Convert numpy arrays to TensorFlow datasets
feature_dataset = tf.data.Dataset.from_tensor_slices(x_train_v1)
target_dataset = tf.data.Dataset.from_tensor_slices(y_train_v1)

# Combine features and target into a single dataset
dataset_tr = tf.data.Dataset.zip((feature_dataset, target_dataset))

# Shuffle and batch the dataset
dataset_train_v1 = dataset_tr.shuffle(buffer_size=len(x_train_v1)).batch(10000)


for batch in dataset_train_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (2178, 7, 30)
Target shape: (2178, 1)


In [29]:
# Find indices of 0s
indices_of_zeros = np.where(targets.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones = np.where(targets.numpy() == 1)[0]

In [30]:
indices_of_zeros.shape,indices_of_ones.shape

((559,), (1619,))

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [31]:
pos_features_v1 = inputs.numpy()[indices_of_ones]
neg_features_v1 = inputs.numpy()[indices_of_zeros]

pos_labels_v1 = targets.numpy()[indices_of_ones]
neg_labels_v1 = targets.numpy()[indices_of_zeros]

pos_features_v1.shape,pos_labels_v1.shape,neg_features_v1.shape,neg_labels_v1.shape

((1619, 7, 30), (1619, 1), (559, 7, 30), (559, 1))

In [32]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds_v1 = make_ds(pos_features_v1, pos_labels_v1)
neg_ds_v1 = make_ds(neg_features_v1, neg_labels_v1)

In [33]:
for num, (features, label) in enumerate(pos_ds_v1.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy(), features.shape)

Label sample 0:  [1] (7, 30)
Label sample 1:  [1] (7, 30)
Label sample 2:  [1] (7, 30)
Label sample 3:  [1] (7, 30)
Label sample 4:  [1] (7, 30)
Label sample 5:  [1] (7, 30)
Label sample 6:  [1] (7, 30)
Label sample 7:  [1] (7, 30)
Label sample 8:  [1] (7, 30)
Label sample 9:  [1] (7, 30)


In [34]:
resampled_ds_v1 = tf.data.Dataset.sample_from_datasets([pos_ds_v1, neg_ds_v1], weights=[0.5, 0.5])
resampled_ds_v1 = resampled_ds_v1.batch(64).prefetch(2)

In [35]:
for features, label in resampled_ds_v1.take(1):
  print(label.shape,label.numpy().mean())

(64, 1) 0.453125


In [36]:
x_valid_v1.shape, y_valid_v1.shape, x_valid.shape, y_valid.shape

((365, 7, 30), (365, 1), (365, 30), (359, 1))

#### **3.2 Validation Dataset**

In [37]:
feature_dataset = tf.data.Dataset.from_tensor_slices(x_valid_v1)
target_dataset = tf.data.Dataset.from_tensor_slices(y_valid_v1)

# Combine features and target into a single dataset
dataset_validation_v1 = tf.data.Dataset.zip((feature_dataset, target_dataset))

# Shuffle and batch the dataset
dataset_validation_v1 = dataset_validation_v1.batch(64).prefetch(2)

for batch in dataset_validation_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


#### **3.3 Test Dataset**

In [38]:
dataset_test_v1 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test_v1.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


## Version 2

In [39]:
dataset_validation_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10_000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation_v2.take(1):
    inputs_val_v2, targets_val_v2 = batch

print("Input shape:", inputs_val_v2.numpy().shape)
print("Target shape:", targets_val_v2.numpy().shape)

Input shape: (359, 7, 30)
Target shape: (359, 1)


In [40]:
dataset_train_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_train,
                                                                    y_train,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=10000,
                                                                    shuffle=True
                                                                 )


for batch in dataset_train_v2.take(1):
    inputs_tr_v2, targets_tr_v2 = batch

print("Input shape:", inputs_tr_v2.numpy().shape)
print("Target shape:", targets_tr_v2.numpy().shape)

Input shape: (2184, 7, 30)
Target shape: (2184, 1)


In [41]:
# Merge the arrays along the first axis
all_train_v2 = np.concatenate((inputs_val_v2, inputs_tr_v2), axis=0)
all_target_v2 = np.concatenate((targets_val_v2, targets_tr_v2), axis=0)
all_train_v2.shape, all_target_v2.shape

((2543, 7, 30), (2543, 1))

In [42]:
all_train.shape[0]-365, all_train.shape

(2178, (2543, 7, 30))

In [43]:
x_train_v2, x_valid_v2, y_train_v2, y_valid_v2 = train_test_split(all_train, all_target, test_size= 0.2, random_state=42, shuffle=True, stratify=all_target)
y_train_v2 = y_train_v2.reshape(-1,1)
y_valid_v2 = y_valid_v2.reshape(-1,1)

print(x_train_v2.shape, y_train_v2.shape)
print(x_valid_v2.shape, y_valid_v2.shape)
# Test Data
x_test_v2 = test_final.iloc[:,:-1].values
y_test_v2 = test_final.iloc[past-1:,-1].values.reshape(-1,1)
print(x_test_v1.shape, y_test_v1.shape)

(2034, 7, 30) (2034, 1)
(509, 7, 30) (509, 1)
(736, 30) (730, 1)


####  **3.1 Train Dataset**

In [44]:
# Convert numpy arrays to TensorFlow datasets
feature_dataset_v2 = tf.data.Dataset.from_tensor_slices(x_train_v2)
target_dataset_v2 = tf.data.Dataset.from_tensor_slices(y_train_v2)

# Combine features and target into a single dataset
dataset_tr_v2 = tf.data.Dataset.zip((feature_dataset_v2, target_dataset_v2))

# Shuffle and batch the dataset
dataset_train_v2 = dataset_tr_v2.shuffle(buffer_size=len(x_train_v2)).batch(10000)


for batch in dataset_train_v2.take(1):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

Input shape: (2034, 7, 30)
Target shape: (2034, 1)


In [45]:
# Find indices of 0s
indices_of_zeros_v2 = np.where(targets_v2.numpy() == 0)[0]

# Find indices of 1s
indices_of_ones_v2 = np.where(targets_v2.numpy() == 1)[0]

In [46]:
indices_of_zeros_v2.shape,indices_of_ones_v2.shape

((524,), (1510,))

* **TRAIN DATASET - INFINITE SAMPLING**

* **GOAL: Oversample the minority class**
A related approach would be to resample the dataset by oversampling the minority class.

In [47]:
pos_features_v2 = inputs_v2.numpy()[indices_of_ones_v2]
neg_features_v2 = inputs_v2.numpy()[indices_of_zeros_v2]

pos_labels_v2 = targets_v2.numpy()[indices_of_ones_v2]
neg_labels_v2 = targets_v2.numpy()[indices_of_zeros_v2]

pos_features_v2.shape,pos_labels_v2.shape,neg_features_v2.shape,neg_labels_v2.shape

((1510, 7, 30), (1510, 1), (524, 7, 30), (524, 1))

In [48]:
BUFFER_SIZE = 100000

def make_ds(features_v2, labels_v2):
  ds_v2 = tf.data.Dataset.from_tensor_slices((features_v2, labels_v2))#.cache()
  ds_v2 = ds_v2.shuffle(BUFFER_SIZE).repeat()
  return ds_v2

pos_ds_v2 = make_ds(pos_features_v2, pos_labels_v2)
neg_ds_v2 = make_ds(neg_features_v2, neg_labels_v2)

In [49]:
for num, (features, label) in enumerate(pos_ds_v2.take(10)):
  #print("Features:\n", features.numpy())
  #print()
  print(f"Label sample {num}: ", label.numpy(), features.shape)

Label sample 0:  [1] (7, 30)
Label sample 1:  [1] (7, 30)
Label sample 2:  [1] (7, 30)
Label sample 3:  [1] (7, 30)
Label sample 4:  [1] (7, 30)
Label sample 5:  [1] (7, 30)
Label sample 6:  [1] (7, 30)
Label sample 7:  [1] (7, 30)
Label sample 8:  [1] (7, 30)
Label sample 9:  [1] (7, 30)


In [50]:
resampled_ds_v2 = tf.data.Dataset.sample_from_datasets([pos_ds_v2, neg_ds_v2], weights=[0.5, 0.5])
resampled_ds_v2 = resampled_ds_v2.batch(64).prefetch(2)

In [51]:
for features, label in resampled_ds_v2.take(1):
  print(label.shape,label.numpy().mean())

(64, 1) 0.46875


In [52]:
x_valid_v2.shape, y_valid_v2.shape, x_valid_v1.shape, y_valid_v1.shape

((509, 7, 30), (509, 1), (365, 7, 30), (365, 1))

#### **3.2 Validation Dataset**

In [53]:
feature_dataset_v2 = tf.data.Dataset.from_tensor_slices(x_valid_v2)
target_dataset_v2 = tf.data.Dataset.from_tensor_slices(y_valid_v2)

# Combine features and target into a single dataset
dataset_validation_v2 = tf.data.Dataset.zip((feature_dataset_v2, target_dataset_v2))

# Shuffle and batch the dataset
dataset_validation_v2 = dataset_validation_v2.batch(64).prefetch(2)

for batch in dataset_validation_v2.take(1):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


#### **3.3 Test Dataset**

In [54]:
dataset_test_v2 = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_test,
                                                                    y_test,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=64,
                                                                    shuffle=False
                                                                 )


for batch in dataset_test_v2.take(1):
    inputs_v2, targets_v2 = batch

print("Input shape:", inputs_v2.numpy().shape)
print("Target shape:", targets_v2.numpy().shape)

Input shape: (64, 7, 30)
Target shape: (64, 1)


## 4.0 NN Models:

In [55]:
def plot_metrics(history):
  metrics = ['loss', 'auc']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0,1])
    else:
      plt.ylim([0,1])

    plt.legend()

def plot_cm(labels, predictions, threshold=0.5):
  cm = confusion_matrix(labels, predictions > threshold)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(threshold))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [56]:
step = 1

past = 7
future = 0
learning_rate = 0.01
batch_size = 64
epochs = 101

In [57]:
METRICS = [
          keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
          ]

### 4.1 LSTM v0

In [ ]:
def make_model(metrics=METRICS, units=[32,32], output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_noised = keras.layers.GaussianNoise(stddev=0.05, name="noise_layer")(data)
  lstm_out = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0")(data_noised)
  lstm_out = keras.layers.LSTM(units[1], name="lstm_1")(lstm_out)
  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(lstm_out)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(units=[64,64])

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Fit the model

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=epochs,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_v2_all_data.csv")

### 4.2 LSTM-Tab v0

In [ ]:
def make_model(metrics=METRICS, units=[32,32],units_tab=[64,32], output_bias=None, gn=0.025, activation="relu", do=0.3):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  lstm_out = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",)(data_noised)
  lstm_out = keras.layers.LSTM(units[1], name="lstm_1")(lstm_out)

  # Tabular Section
  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Concatenate
  x = keras.layers.Concatenate(name="concat")([lstm_out, tabx,data_tabular])
  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_tab_v1")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(units=[64,32],output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=make_model, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 21 finished with value: 0.8943167305236271
    * parameters: {'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 32, 'activation': 'silu', 'gn': 0.09, 'do': 0.30781160547467370}.

 * Trial 41 finished with value: 0.8956648219100326
    * parameters: {'units_0': 128, 'units_1': 64, 'units_tab_0': 128, 'units_tab_1': 32, 'activation': 'selu', 'gn': 0.09, 'do': 0.25}.

* Best is trial 25 with value: 0.8865119909181212.
    * {'units_0': 64,
 'units_1': 32,
 'units_tab_0': 256,
 'units_tab_1': 128,
 'activation': 'selu',
 'gn': 0.08,
 'do': 0.32}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu","mish"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=111, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

#### Fit the model

##### Model 00

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model(units=[64,64], units_tab=[128,32], activation="gelu", gn=0.09, do=0.39, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=epochs,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

##### Model 01

In [ ]:
model_cv1= make_model(units=[128,64], units_tab=[128,128], activation="mish", gn=0.09, do=0.24, output_bias=0)

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3,  factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_v0 = model.predict(dataset_train)
valid_predictions_resampled_v0 = model.predict(dataset_validation)
test_predictions_resampled_v0 = model.predict(dataset_test)

In [ ]:
train_predictions_resampled_v1 = model_cv1.predict(dataset_train)
valid_predictions_resampled_v1 = model_cv1.predict(dataset_validation)
test_predictions_resampled_v1 = model_cv1.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plt.scatter(test_predictions_resampled_v0,test_predictions_resampled_v1)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_v0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_v0.csv")
df_subm["rainfall"] = test_predictions_resampled_v1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_v1.csv")
df_subm["rainfall"] = (test_predictions_resampled_v0+test_predictions_resampled_v1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_v4_all_data_ext_average.csv")
df_subm

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_enhanced_v0_all_data_ext.csv")
df_subm

### 4.3 LSTM-Tab with Transformer v0

In [ ]:
inputs.shape

In [ ]:
def make_model(metrics=METRICS, units=[32,32],units_tab=[64,32], output_bias=None, gn=0.025, activation="relu", do=0.3, lr=5e-4):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  whole_seq_output, final_memory_state_0, final_carry_state_0 = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",return_state=True)(data_noised)
  whole_seq_output, final_memory_state_1, final_carry_state_1 = keras.layers.LSTM(units[1], name="lstm_1",return_state=True)(whole_seq_output)

  print(final_memory_state_0.shape, final_carry_state_0.shape)
  print(final_memory_state_1.shape, final_carry_state_1.shape)

  # Tabular Section
  data_tabular = keras.layers.Concatenate(name="concat_states")([data_tabular,final_memory_state_0,final_carry_state_0,final_memory_state_1,final_carry_state_1])

  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Concatenate
  x = keras.layers.Concatenate(name="concat")([whole_seq_output, tabx,data_tabular])
  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="LSTM_tab_v2")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=lr),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(units=[64,32],output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

In [ ]:
#plot_model(model, show_shapes=True)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","mish","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 85 finished with value: 0.8963743436923514
    * parameters: {'units_0': 64, 'units_1': 32,  'units_tab_0': 128, 'units_tab_1': 32,
 'activation': 'selu', 'gn': 0.1, 'do': 0.44}


  * Trial 69 with value: 0.8962679154250035
    * parameters: {'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64,
 'activation': 'gelu', 'gn': 0.05, 'do': 0.32}

##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              'activation': trial.suggest_categorical('activation', ["relu","mish","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Trial 91 with value: 0.902684004334417
    * parameters: {'units_0': 128,
 'units_1': 64,
 'units_tab_0': 256,
 'units_tab_1': 64,
 'activation': 'mish',
 'gn': 0.09,
 'do': 0.2}

#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
# MODEL 00 'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64, 'activation': 'gelu', 'gn': 0.05, 'do': 0.32
model= make_model(units=[64,32], units_tab=[128,32], activation="selu", gn=0.10, do=0.44, output_bias=0) # Model v3 (best 0.86323)
model= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)
# MODEL 01
model= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv0= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)

resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation
                              )

model_cv0.evaluate(dataset_validation)

In [ ]:
plot_metrics(resampled_history_v0)

In [ ]:
model_cv1= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0, lr=5e-4) # Model v3 (best 0.86323)

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3,  factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

In [ ]:
plt.scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_lstm_tab_enhanced_v0_all_data_ext.csv")
df_subm

### 4.3 LSTM-Tab with Transformer v1 - Last Work

In [ ]:
inputs.shape

In [ ]:
def make_model_cnn_v4(input_shape, metrics=None, filters=32, kernel_size=3, kr=0.01, sdo=0.2,
                    units_dense=128, gn=0.025, activation="relu", do=0.3, strides=2):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)

    # --- Convolutional Block 1 ---
    cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_1")(data_noised)
    cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)
    # NO MaxPooling

    # --- Convolutional Block 2 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_2")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_2")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_2")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_2")(cnn_out)
    # NO MaxPooling

   # --- Convolutional Block 3 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_3")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_3")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_3")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_3")(cnn_out)
    # NO MaxPooling

    # --- Pooling and Concatenation ---
    cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
    cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)
    x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

    # --- Dense Layer ---
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v4")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]),strides=2, metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 66 with value: 0.8893146019582802
    * {'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36}


##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.0.9343330769888062
    * {'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26}


##### Optimization 02

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v4, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v4, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

In [ ]:
cat_params

*  Best is trial 4 with value: 0.8672805848378368
    * {'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu', 'gn': 0.02, 'do': 0.26}


#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=2201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v4(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu','gn': 0.02,'do': 0.26})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv1,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_02.csv")

### 4.3 LSTM-Tab with Transformer v2 - Last Work

In [64]:
inputs.shape

TensorShape([64, 7, 30])

In [65]:
def make_model_cnn_v5(input_shape, metrics=None, filters=32, kernel_size=3, kr=0.01, sdo=0.2,
                    units_dense=128, gn=0.025, activation="relu", do=0.3, strides=2,
                      gru_units = 64):
    """
    Improved 1D CNN model - Strided Convolutions instead of MaxPooling.
    """
    if metrics is None:
        metrics = ['accuracy']

    data = keras.layers.Input(shape=input_shape, name="input_layer")
    data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
    data_tabular = data_noised[:, 6, :]

    ############################# Convolutional Section #############################
    # --- Convolutional Block 1 ---
    cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_1")(data_noised)
    cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)
    # NO MaxPooling

    # --- Convolutional Block 2 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 2, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                    kernel_regularizer=keras.regularizers.l2(kr),
                                    padding="same", activation=None, name="cnn_2")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_2")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_2")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_2")(cnn_out)
    # NO MaxPooling

   # --- Convolutional Block 3 ---
    cnn_out = keras.layers.Conv1D(filters=filters * 4, kernel_size=kernel_size, strides=strides,  # Stride = 2
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_3")(cnn_out)
    cnn_out = keras.layers.LayerNormalization(name="ln_3")(cnn_out)
    cnn_out = keras.layers.Activation("relu", name="act_3")(cnn_out)
    cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_3")(cnn_out)
    # NO MaxPooling

    # --- Pooling and Concatenation ---
    cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
    cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)
    x_conv = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

    ############################# Recursive Section #############################
    whole_seq_output, final_memory_state_00 = keras.layers.GRU(gru_units, return_sequences=True, name="gru_00",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(data_noised)
    whole_seq_output, final_memory_state_01 = keras.layers.GRU(gru_units, return_sequences=True, name="gru_01",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(whole_seq_output)
    whole_seq_output, final_memory_state_10 = keras.layers.GRU(int(gru_units/2), name="gru_1",return_state=True,recurrent_dropout=do,kernel_regularizer=keras.regularizers.l2(kr))(whole_seq_output)

    x_states = keras.layers.Concatenate(name="concat_recursive_states")([final_memory_state_00, final_memory_state_01, final_memory_state_10])
    ############################# Final Dense Section #############################
    # --- Dense Layer ---

    x = keras.layers.Concatenate(name="concat_dense")([x_conv, whole_seq_output,x_states,data_tabular])
    x = keras.layers.Dense(units_dense, name="dense_1", kernel_regularizer=keras.regularizers.l2(kr))(x)
    x = keras.layers.BatchNormalization(name="batch_dense")(x)
    x = keras.layers.Activation(activation, name="act_dense")(x)
    x = keras.layers.Dropout(do, name="do_dense")(x)

    # --- Output Layer ---
    outputs = keras.layers.Dense(1, activation='sigmoid', name="output")(x)

    # --- Model Creation and Compilation ---
    model = keras.Model(inputs=data, outputs=outputs, name="cnn_v4")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=5e-4),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

model= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]),strides=2, metrics=METRICS)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

Model: "cnn_v4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 7, 30)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ noise_layer               │ (None, 7, 30)          │              0 │ input_layer[0][0]      │
│ (GaussianNoise)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cnn_1 (Conv1D)            │ (None, 4, 32)          │          2,912 │ noise_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ln_1 (LayerNormalization) │ (None, 4, 32)          │             64 │ cnn_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ act_1 (Activation)        │ (None, 4, 32)          │              0 │ ln_1[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sdo_1 (SpatialDropout1D)  │ (None, 4, 32)          │              0 │ act_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cnn_2 (Conv1D)            │ (None, 2, 64)          │          6,208 │ sdo_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ln_2 (LayerNormalization) │ (None, 2, 64)          │            128 │ cnn_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ act_2 (Activation)        │ (None, 2, 64)          │              0 │ ln_2[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sdo_2 (SpatialDropout1D)  │ (None, 2, 64)          │              0 │ act_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cnn_3 (Conv1D)            │ (None, 1, 128)         │         24,704 │ sdo_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ln_3 (LayerNormalization) │ (None, 1, 128)         │            256 │ cnn_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ act_3 (Activation)        │ (None, 1, 128)         │              0 │ ln_3[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_00 (GRU)              │ [(None, 7, 64), (None, │         18,432 │ noise_layer[0][0]      │
│                           │ 64)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sdo_3 (SpatialDropout1D)  │ (None, 1, 128)         │              0 │ act_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_01 (GRU)              │ [(None, 7, 64), (None, │         24,960 │ gru_00[0][0]           │
│                           │ 64)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pool_final        │ (None, 128)            │              0 │ sdo_3[0][0]            │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pool_final       

 Total params: 149,025 (582.13 KB)

 Trainable params: 148,769 (581.13 KB)

 Non-trainable params: 256 (1.00 KB)

#### Optuna Optimization

##### Optimization 00

In [66]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [67]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [68]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

[I 2025-03-19 23:39:28,535] A new study created in memory with name: no-name-31890fd0-b1ba-4eed-a4a3-7256f4325f4d


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - auc: 0.8535 - cross entropy: 0.4907 - loss: 1.6710 - prc: 0.8295 - val_auc: 0.8897 - val_cross entropy: 0.4959 - val_loss: 1.4471 - val_prc: 0.9431 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8725 - cross entropy: 0.4615 - loss: 1.3814 - prc: 0.8547 - val_auc: 0.8863 - val_cross entropy: 0.5398 - val_loss: 1.3749 - val_prc: 0.9394 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8787 - cross entropy: 0.4481 - loss: 1.2594 - prc: 0.8656 - val_auc: 0.8868 - val_cross entropy: 0.4561 - val_loss: 1.2023 - val_prc: 0.9407 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8728 - cross entropy: 0.4604 - loss: 1.1970 - prc: 0.8587 - val_auc: 0.8909 - val_cross entropy: 0.4457 - val_loss: 1.1547 - val_prc: 0.9435 - learning_rate: 2.5000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8760 - cross entro

[I 2025-03-19 23:40:15,186] Trial 0 finished with value: 0.8964098197814673 and parameters: {'kernel_size': 2, 'strides': 2, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.31, 'kr': 0.0015371819329827346, 'activation': 'elu', 'gn': 0.08, 'do': 0.30000000000000004}. Best is trial 0 with value: 0.8964098197814673.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - auc: 0.7340 - cross entropy: 0.6161 - loss: 24.7336 - prc: 0.7712 - val_auc: 0.8615 - val_cross entropy: 0.5576 - val_loss: 17.0006 - val_prc: 0.9215 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - auc: 0.8497 - cross entropy: 0.4838 - loss: 15.0472 - prc: 0.8261 - val_auc: 0.8776 - val_cross entropy: 0.5202 - val_loss: 10.3616 - val_prc: 0.9324 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8621 - cross entropy: 0.4629 - loss: 9.1746 - prc: 0.8499 - val_auc: 0.8814 - val_cross entropy: 0.4445 - val_loss: 6.3319 - val_prc: 0.9354 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8755 - cross entropy: 0.4457 - loss: 5.6612 - prc: 0.8556 - val_auc: 0.8830 - val_cross entropy: 0.4639 - val_loss: 4.0082 - val_prc: 0.9373 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - auc: 0.8829 - cross e

[I 2025-03-19 23:40:54,623] Trial 1 finished with value: 0.8901305520079466 and parameters: {'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.42000000000000004, 'kr': 0.0547331729042021, 'activation': 'relu', 'gn': 0.09, 'do': 0.2}. Best is trial 0 with value: 0.8964098197814673.


...

Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - auc: 0.8058 - cross entropy: 0.5657 - loss: 1.4713 - prc: 0.8282 - val_auc: 0.8929 - val_cross entropy: 0.5884 - val_loss: 1.3459 - val_prc: 0.9438 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8655 - cross entropy: 0.4749 - loss: 1.2088 - prc: 0.8414 - val_auc: 0.8962 - val_cross entropy: 0.4457 - val_loss: 1.1158 - val_prc: 0.9452 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8691 - cross entropy: 0.4681 - loss: 1.1203 - prc: 0.8518 - val_auc: 0.8908 - val_cross entropy: 0.4312 - val_loss: 1.0348 - val_prc: 0.9417 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - auc: 0.8783 - cross entropy: 0.4465 - loss: 1.0362 - prc: 0.8523 - val_auc: 0.8936 - val_cross entropy: 0.4845 - val_loss: 1.0362 - val_prc: 0.9445 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8803 - cross entro

[I 2025-03-20 00:58:03,584] Trial 99 finished with value: 0.8952745849297574 and parameters: {'kernel_size': 2, 'strides': 1, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.30000000000000004, 'kr': 0.001213515085565857, 'activation': 'elu', 'gn': 0.02, 'do': 0.28}. Best is trial 23 with value: 0.8993543351780899.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - auc: 0.8033 - cross entropy: 0.5716 - loss: 1.3922 - prc: 0.8227 - val_auc: 0.8933 - val_cross entropy: 0.5872 - val_loss: 1.2722 - val_prc: 0.9440 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - auc: 0.8581 - cross entropy: 0.4886 - loss: 1.1536 - prc: 0.8339 - val_auc: 0.8918 - val_cross entropy: 0.5434 - val_loss: 1.1539 - val_prc: 0.9429 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8752 - cross entropy: 0.4560 - loss: 1.0513 - prc: 0.8709 - val_auc: 0.8922 - val_cross entropy: 0.4648 - val_loss: 1.0184 - val_prc: 0.9435 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - auc: 0.8663 - cross entropy: 0.4781 - loss: 1.0256 - prc: 0.8369 - val_auc: 0.8949 - val_cross entropy: 0.4939 - val_loss: 1.0237 - val_prc: 0.9459 - learning_rate: 2.5000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8866 - cross entro

[I 2025-03-20 00:58:49,095] Trial 100 finished with value: 0.8964098197814673 and parameters: {'kernel_size': 2, 'strides': 1, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.36, 'kr': 0.0010896874879934837, 'activation': 'elu', 'gn': 0.01, 'do': 0.31}. Best is trial 23 with value: 0.8993543351780899.


{'kernel_size': 2,
 'strides': 2,
 'units_dense': 256,
 'gru_units': 64,
 'sdo': 0.32,
 'kr': 0.002801887553503313,
 'activation': 'leaky_relu',
 'gn': 0.09,
 'do': 0.27}

In [69]:
cat_params

{'kernel_size': 2,
 'strides': 2,
 'units_dense': 256,
 'gru_units': 64,
 'sdo': 0.32,
 'kr': 0.002801887553503313,
 'activation': 'leaky_relu',
 'gn': 0.09,
 'do': 0.27}

*  Best is trial 66 with value: 0.8893146019582802
    * {'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36}


##### Optimization 01

In [70]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [71]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [72]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

[I 2025-03-20 00:58:49,200] A new study created in memory with name: no-name-7381716b-4a7f-4124-baa4-97bd858bdb3b


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - auc: 0.8031 - cross entropy: 0.5416 - loss: 5.4535 - prc: 0.8192 - val_auc: 0.8885 - val_cross entropy: 0.6036 - val_loss: 4.3961 - val_prc: 0.9460 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - auc: 0.8711 - cross entropy: 0.4450 - loss: 3.9494 - prc: 0.8647 - val_auc: 0.8911 - val_cross entropy: 0.4471 - val_loss: 3.1965 - val_prc: 0.9444 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8704 - cross entropy: 0.4591 - loss: 3.0125 - prc: 0.8499 - val_auc: 0.8894 - val_cross entropy: 0.4503 - val_loss: 2.4914 - val_prc: 0.9438 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8788 - cross entropy: 0.4424 - loss: 2.3499 - prc: 0.8672 - val_auc: 0.8850 - val_cross entropy: 0.4475 - val_loss: 2.0041 - val_prc: 0.9380 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8903 - cross entro

[I 2025-03-20 00:59:31,518] Trial 0 finished with value: 0.8901409541511772 and parameters: {'kernel_size': 3, 'strides': 1, 'units_dense': 128, 'gru_units': 64, 'sdo': 0.25, 'kr': 0.009012814118281249, 'activation': 'relu', 'gn': 0.08, 'do': 0.22}. Best is trial 0 with value: 0.8901409541511772.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - auc: 0.7500 - cross entropy: 0.6081 - loss: 9.7543 - prc: 0.7759 - val_auc: 0.8804 - val_cross entropy: 0.5348 - val_loss: 7.3249 - val_prc: 0.9455 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8363 - cross entropy: 0.4952 - loss: 6.6822 - prc: 0.8241 - val_auc: 0.8898 - val_cross entropy: 0.4518 - val_loss: 5.0678 - val_prc: 0.9498 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8651 - cross entropy: 0.4587 - loss: 4.6722 - prc: 0.8374 - val_auc: 0.8907 - val_cross entropy: 0.4428 - val_loss: 3.6150 - val_prc: 0.9489 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8739 - cross entropy: 0.4454 - loss: 3.3525 - prc: 0.8640 - val_auc: 0.8906 - val_cross entropy: 0.4174 - val_loss: 2.6365 - val_prc: 0.9468 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8692 - cross entro

[I 2025-03-20 01:00:12,279] Trial 1 finished with value: 0.8911477695167286 and parameters: {'kernel_size': 3, 'strides': 1, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.31, 'kr': 0.020035400386721938, 'activation': 'relu', 'gn': 0.09, 'do': 0.36}. Best is trial 1 with value: 0.8911477695167286.


...

Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - auc: 0.7687 - cross entropy: 0.5955 - loss: 18.9383 - prc: 0.8005 - val_auc: 0.8912 - val_cross entropy: 0.4831 - val_loss: 13.4652 - val_prc: 0.9553 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8580 - cross entropy: 0.4698 - loss: 12.0956 - prc: 0.8517 - val_auc: 0.8945 - val_cross entropy: 0.4152 - val_loss: 8.5756 - val_prc: 0.9531 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8679 - cross entropy: 0.4587 - loss: 7.7641 - prc: 0.8468 - val_auc: 0.8959 - val_cross entropy: 0.4069 - val_loss: 5.5441 - val_prc: 0.9512 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8801 - cross entropy: 0.4399 - loss: 5.0473 - prc: 0.8669 - val_auc: 0.8955 - val_cross entropy: 0.4086 - val_loss: 3.6773 - val_prc: 0.9527 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8744 - cross en

[I 2025-03-20 02:11:57,319] Trial 99 finished with value: 0.8969950433705081 and parameters: {'kernel_size': 2, 'strides': 2, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.4, 'kr': 0.040978132354591655, 'activation': 'elu', 'gn': 0.05, 'do': 0.26}. Best is trial 64 with value: 0.9028423172242874.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - auc: 0.7756 - cross entropy: 0.5888 - loss: 11.4426 - prc: 0.7962 - val_auc: 0.8912 - val_cross entropy: 0.4825 - val_loss: 8.5345 - val_prc: 0.9498 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - auc: 0.8599 - cross entropy: 0.4711 - loss: 7.8004 - prc: 0.8571 - val_auc: 0.8975 - val_cross entropy: 0.3927 - val_loss: 5.8398 - val_prc: 0.9517 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8572 - cross entropy: 0.4780 - loss: 5.4442 - prc: 0.8357 - val_auc: 0.8952 - val_cross entropy: 0.3960 - val_loss: 4.1190 - val_prc: 0.9526 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8734 - cross entropy: 0.4564 - loss: 3.8633 - prc: 0.8644 - val_auc: 0.8989 - val_cross entropy: 0.4077 - val_loss: 2.9977 - val_prc: 0.9526 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8668 - cross entr

[I 2025-03-20 02:12:38,041] Trial 100 finished with value: 0.8991248451053284 and parameters: {'kernel_size': 2, 'strides': 2, 'units_dense': 64, 'gru_units': 64, 'sdo': 0.44, 'kr': 0.023870352470596486, 'activation': 'elu', 'gn': 0.04, 'do': 0.31}. Best is trial 64 with value: 0.9028423172242874.


{'kernel_size': 2,
 'strides': 2,
 'units_dense': 64,
 'gru_units': 64,
 'sdo': 0.4,
 'kr': 0.028176235373792834,
 'activation': 'elu',
 'gn': 0.03,
 'do': 0.29000000000000004}

In [73]:
cat_params

{'kernel_size': 2,
 'strides': 2,
 'units_dense': 64,
 'gru_units': 64,
 'sdo': 0.4,
 'kr': 0.028176235373792834,
 'activation': 'elu',
 'gn': 0.03,
 'do': 0.29000000000000004}

*  Best is trial 4 with value: 0.0.9343330769888062
    * {'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26}


##### Optimization 02

In [74]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn_v5, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),#
              'strides': trial.suggest_categorical('strides', [1,2]),#
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),#
              'gru_units': trial.suggest_categorical('gru_units', [128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),#
              'kr': trial.suggest_float('kr', 0.001, 0.1, log=True),#
              'activation': trial.suggest_categorical('activation', ["relu","elu","gelu","silu","leaky_relu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v2, y_pred)

    return rmse_scores

In [75]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [76]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v2,validation_data=dataset_validation_v2, model_class=make_model_cnn_v5, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

[I 2025-03-20 02:12:38,078] A new study created in memory with name: no-name-bafef1d4-68bf-4324-8162-a70483423a06


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 18s 65ms/step - auc: 0.8310 - cross entropy: 0.5403 - loss: 13.9768 - prc: 0.8452 - val_auc: 0.8813 - val_cross entropy: 0.4230 - val_loss: 9.3812 - val_prc: 0.9462 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - auc: 0.8605 - cross entropy: 0.4951 - loss: 8.4159 - prc: 0.8509 - val_auc: 0.8748 - val_cross entropy: 0.4541 - val_loss: 5.8555 - val_prc: 0.9419 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - auc: 0.8737 - cross entropy: 0.4687 - loss: 5.3032 - prc: 0.8591 - val_auc: 0.8841 - val_cross entropy: 0.4296 - val_loss: 3.8907 - val_prc: 0.9439 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - auc: 0.8715 - cross entropy: 0.4777 - loss: 3.6267 - prc: 0.8676 - val_auc: 0.8778 - val_cross entropy: 0.3564 - val_loss: 2.7406 - val_prc: 0.9395 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - auc: 0.8758 - cross entr

[I 2025-03-20 02:13:26,406] Trial 0 finished with value: 0.8779635688032635 and parameters: {'kernel_size': 3, 'strides': 1, 'units_dense': 256, 'gru_units': 128, 'sdo': 0.25, 'kr': 0.016166944530616463, 'activation': 'elu', 'gn': 0.09, 'do': 0.45}. Best is trial 0 with value: 0.8779635688032635.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 62ms/step - auc: 0.8462 - cross entropy: 0.5129 - loss: 1.7009 - prc: 0.8627 - val_auc: 0.8693 - val_cross entropy: 0.4869 - val_loss: 1.4414 - val_prc: 0.9327 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - auc: 0.8872 - cross entropy: 0.4379 - loss: 1.3518 - prc: 0.8721 - val_auc: 0.8709 - val_cross entropy: 0.4463 - val_loss: 1.2565 - val_prc: 0.9344 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - auc: 0.8825 - cross entropy: 0.4480 - loss: 1.2318 - prc: 0.8786 - val_auc: 0.8748 - val_cross entropy: 0.3746 - val_loss: 1.0895 - val_prc: 0.9398 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - auc: 0.8857 - cross entropy: 0.4383 - loss: 1.1349 - prc: 0.8812 - val_auc: 0.8768 - val_cross entropy: 0.3721 - val_loss: 1.0200 - val_prc: 0.9377 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - auc: 0.8719 - cross entro

[I 2025-03-20 02:14:15,088] Trial 1 finished with value: 0.8786501878104931 and parameters: {'kernel_size': 2, 'strides': 2, 'units_dense': 256, 'gru_units': 128, 'sdo': 0.24000000000000002, 'kr': 0.0012573324413816438, 'activation': 'elu', 'gn': 0.01, 'do': 0.42000000000000004}. Best is trial 1 with value: 0.8786501878104931.


...

Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - auc: 0.8273 - cross entropy: 0.5322 - loss: 7.5242 - prc: 0.8542 - val_auc: 0.8754 - val_cross entropy: 0.4392 - val_loss: 5.7380 - val_prc: 0.9441 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8771 - cross entropy: 0.4499 - loss: 5.3256 - prc: 0.8724 - val_auc: 0.8678 - val_cross entropy: 0.3869 - val_loss: 4.1694 - val_prc: 0.9334 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8743 - cross entropy: 0.4611 - loss: 3.9668 - prc: 0.8680 - val_auc: 0.8766 - val_cross entropy: 0.4085 - val_loss: 3.1988 - val_prc: 0.9408 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc: 0.8817 - cross entropy: 0.4433 - loss: 3.0508 - prc: 0.8697 - val_auc: 0.8759 - val_cross entropy: 0.3574 - val_loss: 2.4889 - val_prc: 0.9389 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8807 - cross entro

[I 2025-03-20 03:40:54,879] Trial 99 finished with value: 0.8787713558705925 and parameters: {'kernel_size': 2, 'strides': 2, 'units_dense': 256, 'gru_units': 64, 'sdo': 0.33, 'kr': 0.01046581676052707, 'activation': 'elu', 'gn': 0.09999999999999999, 'do': 0.33}. Best is trial 97 with value: 0.8861020235066037.


Epoch 1/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - auc: 0.8232 - cross entropy: 0.5217 - loss: 7.5932 - prc: 0.8505 - val_auc: 0.8765 - val_cross entropy: 0.5701 - val_loss: 5.5860 - val_prc: 0.9407 - learning_rate: 5.0000e-04
Epoch 2/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - auc: 0.8753 - cross entropy: 0.4583 - loss: 4.9866 - prc: 0.8693 - val_auc: 0.8772 - val_cross entropy: 0.4831 - val_loss: 3.7912 - val_prc: 0.9411 - learning_rate: 5.0000e-04
Epoch 3/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - auc: 0.8812 - cross entropy: 0.4425 - loss: 3.4603 - prc: 0.8704 - val_auc: 0.8692 - val_cross entropy: 0.4730 - val_loss: 2.7663 - val_prc: 0.9407 - learning_rate: 5.0000e-04
Epoch 4/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - auc: 0.8810 - cross entropy: 0.4414 - loss: 2.5609 - prc: 0.8776 - val_auc: 0.8764 - val_cross entropy: 0.3667 - val_loss: 2.0489 - val_prc: 0.9378 - learning_rate: 5.0000e-04
Epoch 5/101
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - auc: 0.8846 - cross entro

[I 2025-03-20 03:41:53,422] Trial 100 finished with value: 0.8802859566218344 and parameters: {'kernel_size': 2, 'strides': 1, 'units_dense': 128, 'gru_units': 128, 'sdo': 0.23, 'kr': 0.009974556596034412, 'activation': 'elu', 'gn': 0.06999999999999999, 'do': 0.32}. Best is trial 97 with value: 0.8861020235066037.


{'kernel_size': 2,
 'strides': 1,
 'units_dense': 128,
 'gru_units': 128,
 'sdo': 0.23,
 'kr': 0.010294171991957668,
 'activation': 'elu',
 'gn': 0.09,
 'do': 0.32}

In [77]:
cat_params

{'kernel_size': 2,
 'strides': 1,
 'units_dense': 128,
 'gru_units': 128,
 'sdo': 0.23,
 'kr': 0.010294171991957668,
 'activation': 'elu',
 'gn': 0.09,
 'do': 0.32}

*  Best is trial 4 with value: 0.8672805848378368
    * {'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu', 'gn': 0.02, 'do': 0.26}


#### Fit the model 00

In [ ]:
model_cv0= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 1, 'units_dense': 64, 'sdo': 0.43, 'kr': 0.0202, 'activation':'elu', 'gn': 0.07,'do': 0.36})
model_cv0.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=2201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history_v0)

In [ ]:
#plot_model(model, show_shapes=True)

#### Fit the model 01

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv1= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 3, 'strides': 2, 'units_dense': 64, 'sdo': 0.37, 'kr': 0.0040221083422204585, 'activation': 'relu', 'gn': 0.10, 'do': 0.26})

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Fit the model 02

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
model_cv2= make_model_cnn_v5(input_shape=(inputs.shape[1], inputs.shape[2]), metrics=METRICS, **{'kernel_size': 2, 'strides': 2, 'units_dense': 128, 'sdo': 0.31, 'kr': 0.003562788704053626, 'activation': 'leaky_relu','gn': 0.02,'do': 0.26})

resampled_history_v2 = model_cv2.fit(
                              resampled_ds_v2,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=151, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v2
                              )

model_cv2.evaluate(dataset_validation_v2)

In [ ]:
plot_metrics(resampled_history_v2)

#### Evaluate Model:

**MODEL 00**

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

**MODEL 01**

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

**MODEL 02**

In [ ]:
train_predictions_resampled_cv2 = model_cv2.predict(dataset_train_v2)
valid_predictions_resampled_cv2 = model_cv2.predict(dataset_validation_v2)
test_predictions_resampled_cv2 = model_cv2.predict(dataset_test_v2)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].scatter(test_predictions_resampled_cv0,test_predictions_resampled_cv1)
ax[1].scatter(test_predictions_resampled_cv1,test_predictions_resampled_cv2)
plt.show()

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
plot_cm(y_valid_v1, valid_predictions_resampled_cv1)

In [ ]:
plot_cm(y_valid_v2, valid_predictions_resampled_cv2)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape, test_predictions_resampled_cv2.shape

In [ ]:
df_subm["rainfall"] = (test_predictions_resampled_cv0+test_predictions_resampled_cv1+test_predictions_resampled_cv2)/3
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext.csv")
df_subm

In [ ]:
df_subm["rainfall"] = test_predictions_resampled_cv0
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_00.csv")
df_subm["rainfall"] = test_predictions_resampled_cv1
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_01.csv")
df_subm["rainfall"] = test_predictions_resampled_cv2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v3_all_data_ext_02.csv")

### 4.3 CNN Classifier

In [ ]:
def make_model_cnn(metrics=METRICS, filters=64, kernel_size=2, kr=0.01, sdo=0.2,
               units_dense=256, output_bias=None, gn=0.025, activation="relu", do=0.3):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")

  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  # Cnn Layers
  # Layer 0
  cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_0")(data_noised)
  cnn_out = keras.layers.LayerNormalization(name="ln_0")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_0")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_0")(cnn_out)
  cnn_out = keras.layers.MaxPooling1D(pool_size=2, name="max_pool_1")(cnn_out)
  # Layer 1
  cnn_out = keras.layers.Conv1D(filters=int(filters*2), kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_1")(cnn_out)
  cnn_out = keras.layers.LayerNormalization(name="ln_1")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_1")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_1")(cnn_out)

  #outputs
  cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
  cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)

  x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

  # Dense Layer
  x = keras.layers.Dense(units_dense, name="dense_1")(x)
  x = keras.layers.BatchNormalization(name="batch_dense")(x)
  x = keras.layers.Activation(activation, name="act_dense")(x)
  x = keras.layers.Dropout(do, name="do_dense")(x)

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="cnn_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=5e-4),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model_cnn(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn, use_gpu=False, rs=42, fit_scaling=False, epochs=21):

    model_class = model

    params = {
              'filters': trial.suggest_categorical('filters', [16,32,64]),
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),
              'kr': trial.suggest_float('kr', 0.01, 0.1, step=0.01),
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01)
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=make_model_cnn, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

*  Best is trial 66 with value: 0.895310061
    * {'filters': 64,
 'kernel_size': 2,
 'units_dense': 256,
 'do': 0.39,
 'sdo': 0.39,
 'kr': 0.09999999999999999,
 'activation': 'gelu',
 'gn': 0.09999999999999999}


#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model_cnn(**{'filters': 64, 'kernel_size': 2, 'units_dense': 256, 'do': 0.39, 'sdo': 0.39, 'kr': 0.1, 'activation': 'gelu', 'gn': 0.1}) # Model v

model.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=2201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=101, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_v2_all_data_ext.csv")

### 4.4 CNN Classifier - Tab v0

In [ ]:
def make_model_cnn(metrics=METRICS, filters=64, kernel_size=2, kr=0.01, sdo=0.2,
                   units_dense=256, output_bias=None, gn=0.025, activation="relu",
                   do=0.3, units_tab=[128,128], lr=5e-4):

  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]

  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  # Cnn Layers
  # Layer 0
  cnn_out = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_0")(data_noised)
  cnn_out = keras.layers.LayerNormalization(name="ln_cnn_0")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_cnn_0")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_cnn_0")(cnn_out)
  cnn_out = keras.layers.MaxPooling1D(pool_size=2, name="max_pool_cnn_1")(cnn_out)
  # Layer 1
  cnn_out = keras.layers.Conv1D(filters=int(filters*2), kernel_size=kernel_size, strides=1,
                                kernel_regularizer=keras.regularizers.l2(kr),
                                padding="same", activation=None, name="cnn_1")(cnn_out)
  cnn_out = keras.layers.LayerNormalization(name="ln_cnn_1")(cnn_out)
  cnn_out = keras.layers.Activation("relu", name="act_cnn_1")(cnn_out)
  cnn_out = keras.layers.SpatialDropout1D(sdo, name="sdo_cnn_1")(cnn_out)

  #outputs
  cnn_out_ave = keras.layers.GlobalAveragePooling1D(name="average_pool_final")(cnn_out)
  cnn_out_max = keras.layers.GlobalMaxPooling1D(name="max_pool_final")(cnn_out)

  x = keras.layers.Concatenate(name="concat")([cnn_out_ave, cnn_out_max])

  # Dense Layer
  x = keras.layers.Dense(units_dense, name="dense_conv_1")(x)
  x = keras.layers.BatchNormalization(name="batch_dense_conv")(x)
  x = keras.layers.Activation(activation, name="act_dense_conv")(x)
  x = keras.layers.Dropout(do, name="do_dense_conv")(x)

  # Tabular Section
  tabx = keras.layers.Dense(units_tab[0], name="dense_0")(data_tabular)
  tabx = keras.layers.BatchNormalization(name="batch_0")(tabx)
  tabx = keras.layers.Activation(activation, name="act_0")(tabx)
  tabx = keras.layers.Dropout(do, name="do_0")(tabx)
  tabx = keras.layers.Dense(units_tab[1], name="dense_1")(tabx)
  tabx = keras.layers.BatchNormalization(name="batch_1")(tabx)
  tabx = keras.layers.Activation(activation, name="act_1")(tabx)
  tabx = keras.layers.Dropout(do, name="do_1")(tabx)

  # Final Concatenation
  x = keras.layers.Concatenate(name="final_concat")([x, tabx])

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="cnn_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=lr),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model_cnn(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

#### Optuna Optimization

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model_cnn, use_gpu=False, rs=42, fit_scaling=False,epochs=51):

    model_class = model

    params = {
              'filters': trial.suggest_categorical('filters', [32,64,128]),
              'kernel_size': trial.suggest_categorical('kernel_size', [2,3]),
              'units_dense': trial.suggest_categorical('units_dense', [256, 128, 64]),
              'sdo': trial.suggest_float('sdo', 0.20, 0.45, step=0.01),
              'kr': trial.suggest_float('kr', 0.01, 0.1, step=0.01),
              'activation': trial.suggest_categorical('activation', ["relu","selu","gelu","silu"]),
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.20, 0.45, step=0.01),
              'units_tab': [trial.suggest_categorical('units_tab_0', [256, 128, 64]),trial.suggest_categorical('units_tab_1', [128, 64,32])],
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=epochs,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model_cnn, n_trials=151, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

 * Best is trial 119 with value: 0.9003476656733361.
    * parameters: {'filters': 32,
 'kernel_size': 3,
 'units_dense': 256,
 'do': 0.26,
 'sdo': 0.26,
 'kr': 0.08,
 'activation': 'relu',
 'gn': 0.03,
 'units_tab_0': 128,
 'units_tab_1': 64}.


 * Best is trial 129 with value: 0.897686958989641.
    * parameters: {'filters': 64, 'kernel_size': 3, 'units_dense': 64, 'sdo': 0.22, 'kr': 0.05, 'activation': 'selu', 'gn': 0.06, 'do': 0.2, 'units_tab_0': 128, 'units_tab_1': 64}


#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
model= make_model_cnn(lr=5e-4, **{'filters': 32, 'kernel_size': 3, 'units_dense': 256, 'do': 0.26, 'sdo': 0.26, 'kr': 0.08, 'activation': 'relu', 'gn': 0.03, "units_tab":[128, 64]}) # auc: 0.8876
model= make_model_cnn(lr=5e-4, **{'filters': 64, 'kernel_size': 3, 'units_dense': 64, 'do': 0.20, 'sdo': 0.22, 'kr': 0.05, 'activation': 'relu', 'gn': 0.06, "units_tab":[128, 64]}) # auc: 0.8853


model.summary()

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

In [ ]:
resampled_history = model.fit(
                              resampled_ds,
                              epochs=151,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", min_lr=0.00001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")],
                              validation_data=dataset_validation
                              )

In [ ]:
plot_metrics(resampled_history)

#### Evaluate Model:

In [ ]:
train_predictions_resampled = model.predict(dataset_train)
valid_predictions_resampled = model.predict(dataset_validation)
test_predictions_resampled = model.predict(dataset_test)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled)

In [ ]:
df_subm["rainfall"] = test_predictions_resampled
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_cnn_tab_v1_all_data_ext.csv")

### 4.5 Temporal Transformer

In [ ]:
num_samples = 5000
sequence_length = 7
num_features = 30
embed_dim = num_features
head_size = 32
num_heads = 4
gru_units = 64
dropout = 0.1

# 2. Positional Embedding Layer
class TimeSeriesPositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

# 3. Transformer Encoder Layer with GRU
def transformer_encoder_gru(inputs, head_size, num_heads, gru_units, embed_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part (GRU)
    x = keras.layers.GRU(units=gru_units, return_sequences=True)(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Dense(units=embed_dim)(x) # Added Dense Layer
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [ ]:
def make_model(metrics=METRICS, units=[32,32], output_bias=None, gn=0.025, activation="relu", do=0.3, num_transformer_layers=2,
               head_size=32, num_heads=4, gru_units=64):
  """
  Builds a transformer-based binary classification model for time series data.

  Args:
      input_shape: Tuple, the shape of the input data (sequence_length, num_features).
      num_transformer_layers: Integer, the number of transformer encoder layers to stack.
      head_size: Integer, the size of each attention head.
      num_heads: Integer, the number of attention heads.
      gru_units: Integer, the number of GRU units in the feed-forward part.
      dropout: Float, the dropout rate.

  Returns:
      A Keras model.
  """

  tot_obs, sequence_length, num_features = inputs.shape
  embed_dim = inputs.shape[2]

  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  data = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]), name="input_layer")
  data_tabular = data[:, 6, :]
  # LSTM Section
  data_noised = keras.layers.GaussianNoise(stddev=gn, name="noise_layer")(data)
  lstm_out = keras.layers.LSTM(units[0], return_sequences=True, name="lstm_0",)(data_noised)
  lstm_out = keras.layers.LSTM(units[1], name="lstm_1")(lstm_out)
  lstm_out = keras.layers.Dropout(do)(lstm_out)

  # Tabular Section

  x = TimeSeriesPositionalEmbedding(sequence_length, embed_dim)(data)

  # Stack Transformer Layers
  for _ in range(num_transformer_layers):
      x = transformer_encoder_gru(x, head_size, num_heads, gru_units, embed_dim, dropout)

  # Global Pooling and Output Layer
  x_av = keras.layers.GlobalAveragePooling1D()(x)
  x_av = keras.layers.Dropout(do)(x_av)
  # Max Pooling and Output Layer
  x_mx = keras.layers.GlobalMaxPooling1D()(x)
  x_mx = keras.layers.Dropout(do)(x_mx)


  # Concatenate
  x = keras.layers.Concatenate(name="concat")([lstm_out, x_av,x_mx])

  outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias, name="output")(x)

  model = keras.Model(inputs=data, outputs=outputs, name="temp_trans_v0")

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

model= make_model(output_bias=0)

# Reset the bias to zero, since this dataset is balanced.
output_layer = model.layers[-1]
output_layer.bias.assign([0])

model.summary()

In [ ]:
#plot_model(model, show_shapes=True)

#### Optuna Optimization

##### Optimization 00

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False, epochs=101):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'output_bias':0,
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.15, 0.45, step=0.01),
              'num_transformer_layers': trial.suggest_categorical('num_transformer_layers', [2,3]),
              'head_size': trial.suggest_categorical('head_size', [32,64]),
              'num_heads': trial.suggest_categorical('num_heads', [4,8,16]),
              'gru_units': trial.suggest_categorical('gru_units', [32,64,128])
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, verbose=1, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds,validation_data=dataset_validation, model_class=make_model, n_trials=101, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

* Best is trial 91 with value: 0.8922236412657868
  * Parameters: {'units_0': 128, 'units_1': 64, 'gn': 0.08, 'do': 0.43999999999999995, 'num_transformer_layers': 3, 'head_size': 32, 'num_heads': 4, 'gru_units': 128}

##### Optimization 01

In [ ]:
def objective_nn(trial, train_data, validation_data, model=make_model, use_gpu=False, rs=42, fit_scaling=False):

    model_class = model

    params = {
              'units': [trial.suggest_categorical('units_0', [128,64]),trial.suggest_categorical('units_1', [64,32])],
              'output_bias':0,
              'gn': trial.suggest_float('gn', 0.01, 0.1, step=0.01),
              'do': trial.suggest_float('do', 0.15, 0.45, step=0.01),
              'num_transformer_layers': trial.suggest_categorical('num_transformer_layers', [2,3]),
              'head_size': trial.suggest_categorical('head_size', [32,64]),
              'num_heads': trial.suggest_categorical('num_heads', [4,8,16]),
              'gru_units': trial.suggest_categorical('gru_units', [32,64,128])
              }

    auc_score = []

    keras.utils.set_random_seed(rs)
    model = model_class(**params)

    # Fit the model
    model.fit(train_data,
              validation_data=validation_data,
              epochs=101,
              steps_per_epoch=52,
              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, verbose=1, monitor="val_auc", min_lr=0.000001, mode="max"),
                         keras.callbacks.EarlyStopping(patience=21, restore_best_weights=True, monitor="val_auc",
                                                       start_from_epoch=3, mode="max")]
              )

    # Make predictions on the validation set
    y_pred = model.predict(validation_data)

    # Calculate the RMSE for the current fold
    rmse_scores = roc_auc_score(y_valid_v1, y_pred)

    return rmse_scores

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(train_data, validation_data, model_class, n_trials, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    study.optimize(lambda trial: objective_nn(trial, train_data=train_data, validation_data=validation_data, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

In [ ]:
nn_study = tune_hyperparameters(train_data=resampled_ds_v1,validation_data=dataset_validation_v1, model_class=make_model, n_trials=111, use_gpu=False)
#save_results(cat_study, TabNetClassifier, "tabnet_ext")
cat_params = nn_study.best_params
cat_params

#### Fit the model

In [ ]:
# model= make_model(units=[64,64], units_tab=[64,32], activation="silu", gn=0.09, do=0.31, output_bias=0) # Model v2 (Score 0.85787)
# model= make_model(units=[128,64], units_tab=[128,32], activation="selu", gn=0.09, do=0.25, output_bias=0) # Model v1 (best 0.86323)
# MODEL 00 'units_0': 64, 'units_1': 64, 'units_tab_0': 64, 'units_tab_1': 64, 'activation': 'gelu', 'gn': 0.05, 'do': 0.32
model= make_model(units=[64,32], units_tab=[128,32], activation="selu", gn=0.10, do=0.44, output_bias=0) # Model v3 (best 0.86323)
model= make_model(units=[64,64], units_tab=[64,64], activation="gelu", gn=0.05, do=0.32, output_bias=0) # Model v3 (best 0.86323)
# MODEL 01
model= make_model(units=[128,64], units_tab=[256,64], activation="mish", gn=0.09, do=0.20, output_bias=0) # Model v3 (best 0.86323)

In [ ]:
pos = pos_features.shape[0]
neg = neg_features.shape[0]
print(pos,neg)
resampled_steps_per_epoch = int(np.ceil(2.0*pos/batch_size))
resampled_steps_per_epoch

#{'units_0': 128, 'units_1': 64, 'gn': 0.08, 'do': 0.43999999999999995, 'num_transformer_layers': 3, 'head_size': 32, 'num_heads': 4, 'gru_units': 128}

##### Model 00

In [ ]:
model_cv0= make_model(metrics=METRICS, units=[128,64], output_bias=0, gn=0.08, do=0.44, num_transformer_layers=3,
                        head_size=32, num_heads=4, gru_units=128) # Model v0

resampled_history_v0 = model_cv0.fit(
                              resampled_ds,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation
                              )

model_cv0.evaluate(dataset_validation)

In [ ]:
plot_metrics(resampled_history_v0)

##### Model 01

In [ ]:
model_cv1= make_model(metrics=METRICS, units=[64,64], output_bias=0, gn=0.03, do=0.43, num_transformer_layers=2,
                        head_size=64, num_heads=16, gru_units=128) # Model v0

resampled_history_v1 = model_cv1.fit(
                              resampled_ds_v1,
                              epochs=201,
                              steps_per_epoch=52,
                              callbacks=[keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_auc", mode="max", min_lr=0.000001, verbose=1),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max", verbose=1)],
                              validation_data=dataset_validation_v1
                              )

model_cv1.evaluate(dataset_validation_v1)

In [ ]:
plot_metrics(resampled_history_v1)

#### Evaluate Model:

In [ ]:
train_predictions_resampled_cv0 = model_cv0.predict(dataset_train)
valid_predictions_resampled_cv0 = model_cv0.predict(dataset_validation)
test_predictions_resampled_cv0 = model_cv0.predict(dataset_test)

In [ ]:
# df_subm["rainfall"] = test_predictions_resampled_cv0
# df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v0_all_data_ext.csv")
df_subm_v0 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v0_all_data_ext.csv", index_col=0)
df_subm_v0

In [ ]:
train_predictions_resampled_cv1 = model_cv1.predict(dataset_train_v1)
valid_predictions_resampled_cv1 = model_cv1.predict(dataset_validation_v1)
test_predictions_resampled_cv1 = model_cv1.predict(dataset_test_v1)

In [ ]:
df_subm_v1 = df_subm.copy()
df_subm_v1["rainfall"] = test_predictions_resampled_cv1
df_subm_v1.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v1_all_data_ext.csv")
df_subm_v1 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v1_all_data_ext.csv", index_col=0)
df_subm_v1

In [ ]:
plt.scatter(df_subm_v0,df_subm_v1)

In [ ]:
plot_cm(y_valid, valid_predictions_resampled_cv0)

In [ ]:
#plot_cm(y_valid_v1, dataset_validation_v1)
test_predictions_resampled_cv0.shape, test_predictions_resampled_cv1.shape

In [ ]:
df_subm["rainfall"] = (df_subm_v1+df_subm_v0)/2
df_subm.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S5E3/submission_transformer_enhanced_v_ensemble_all_data_ext.csv")
df_subm

### 4.3 CNN Classifier

In [ ]:
%%capture
!pip install sktime

In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier

METRICS = [
          keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
          keras.metrics.AUC(name='auc'),
          keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
          ]

In [ ]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(10_000).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())
  print(label.numpy().shape)
  print(features.numpy().shape)

In [ ]:
dataset_validation = keras.preprocessing.timeseries_dataset_from_array(
                                                                    x_valid,
                                                                    y_valid,
                                                                    sequence_length=7,
                                                                    sampling_rate=step,
                                                                    batch_size=2000,
                                                                    shuffle=False
                                                                 )


for batch in dataset_validation.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
train = np.concatenate((inputs, features), axis=0)
y = np.concatenate((targets, label), axis=0)
train.shape, y.shape

In [ ]:
call_bk = [keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3, monitor="val_auc", min_lr=0.000001),
                                         keras.callbacks.EarlyStopping(patience=31, restore_best_weights=True, monitor="val_auc",
                                                                      start_from_epoch=3, mode="max")]

model = GRUFCNNClassifier( hidden_dim=128, gru_layers=2, verbose=True)





#CNNClassifier(n_epochs=8, batch_size=64, kernel_size=3, avg_pool_size=2, n_conv_layers=2, callbacks=call_bk,
#                      verbose=True, loss=keras.losses.BinaryCrossentropy(), metrics=METRICS, random_state=42,
#                      activation='sigmoid', use_bias=True, optimizer=keras.optimizers.Adam(learning_rate=1e-3), filter_sizes=None, padding='auto')

In [ ]:
y_prob_pred = model.fit_predict_proba(train, y, cv=5)

In [ ]:
import tensorflow as tf
from tensorflow import keras

class TimeSeriesPositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, activation='gelu')(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

# Example Usage
sequence_length = 7 # Example sequence length
embed_dim = 30 # Example embedding dimension.
head_size = 32
num_heads = 4
ff_dim = 128
dropout = 0.1

inputs = keras.layers.Input(shape=(sequence_length, embed_dim))
x = TimeSeriesPositionalEmbedding(sequence_length, embed_dim)(inputs)
x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

## Test SKTime:

In [ ]:
%%capture
!pip install sktime

In [ ]:
from sktime.datasets import load_italy_power_demand

In [ ]:
# Increase display width
pd.set_option("display.width", 1000)

## 2.1 Panel data - sktime data formats

Panel is an abstract data type where the values are observed for:

* instance, e.g., patient
* variable, e.g., blood pressure, body temperature of the patient
* time/index, e.g., January 12, 2023 (usually but not necessarily a time index!)
One value X is: "patient 'A' had blood pressure 'X' on January 12, 2023"

Time series classification, regression, clustering: slices Panel data by instance

Preferred format 1: pd.DataFrame with 2-level MultiIndex, (instance, time) and columns: variables

Preferred format 2: 3D np.ndarray with index (instance, variable, time)

* sktime supports and recognizes multiple data formats for convenience and internal use, e.g., dask, xarray
* abstract data type = "scitype"; in-memory specification = "mtype"
* More information in tutorial on in-memory data representations and data loading

## 2.1.1 Preferred format 1 - pd-multiindex specification

pd-multiindex = pd.DataFrame with 2-level MultiIndex, (instance, time) and columns: variables

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in pd-multiindex mtype
X, _ = load_italy_power_demand(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["total_power_demand"]
X.index.names = ["day_ID", "hour_of_day"]

The Italy power demand dataset has:

* 1096 individual time series instances = single days of total power demand (mean subtracted)
* one single variable per time series instances, total_power_demand
    * total power demand on that day, in that hourly period
    * Since there's only one column, it is a univariate dataset
* individual time series are observed at 24 time (period) points (the same number for all instances)

In the dataset, days are jumbled and of different scope (independent sampling).

* considered independent - because hour_of_day in one sample doesn't affect hour_of_day in another
* for task, e.g., "identify season or weekday/weekend from pattern"

In [ ]:
X

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in pd-multiindex mtype
X, _ = load_basic_motions(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["accel_1", "accel_2", "accel_3", "gyro_1", "gyro_2", "gyro_3"]
X.index.names = ["trial_no", "timepoint"]

The basic motions dataset has:

* 80 individual time series instances = trials = person engaging in an activity like running, badminton, etc.
* six variables per time series instance, dim_0 to dim_5 (renamed according to the values they represent)
  * 3 accelerometer and 3 gyrometer measurements
  * hence a multivariate dataset
* individual time series are observed at 100 time points (the same number for all instances)

In [ ]:
# The outermost index represents the instance number
# whereas the inner index represents the index of the particular index
# within that instance.
X

In [ ]:
# Select:
# * the fourth variable (gyroscope 1)
# * of the first instance (trial 1 = 0 in python)
# * values at all 100 timestamps
#
X.loc[0, "gyro_1"]

In [ ]:
X.loc[0, "gyro_1"].plot()

## 2.1.2 preferred format 2 - numpy3D

    numpy3D = 3D np.ndarray with index (instance, variable, time)

instance/time index is interpreted as integer

IMPORTANT: unlike pd-multiindex, this assumes:

* all individual series have the same length

* all individual series have the same index

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in numpy mtype
X, _ = load_italy_power_demand(return_type="numpy3D")

In [ ]:
# (num_instances, num_variables, length)
X.shape

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in numpy mtype
X, _ = load_basic_motions(return_type="numpy3D")

In [ ]:
# (num_instances, num_variables, length)
X.shape

## 2.2.3 Time Series Classification - deployment vignette
Basic deployment vignette for TSC:

load/setup training data, X in a Panel (more specifically numpy3D) format, y as 1D np.ndarray

load/setup new data for prediction (can be done after 3 too)

specify the classifier using sklearn-like syntax

fit classifier to training data, fit(X, y)

predict labels on new data, predict(X_new)

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_new, _ = load_italy_power_demand(split="test", return_type="numpy3D")

In [ ]:
# this is in numpy3D format, but could also be pd-multiindex or other
X_train.shape, X_new.shape

In [ ]:
# y is a 1D np.ndarray of labels - same length as number of instances in X_train
y_train.shape

In [ ]:
# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3)

# example 2 - custom distance:
# 3-nearest neighbour classifier with Euclidean distance (on flattened time series)
# (requires scipy)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=eucl_dist)

In [ ]:
# all classifiers is scikit-learn / scikit-base compatible!
# nested parameter interface via get_params, set_params
clf.get_params()

In [ ]:
# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

In [ ]:
# the classifier is now fitted
clf.is_fitted

In [ ]:
# and we can inspect fitted parameters if we like
clf.get_fitted_params()

In [ ]:
# step 5 - predict labels on new data
y_pred = clf.predict(X_new)
y_prob = clf.predict_proba(X_new)

In [ ]:
# predictions and unique counts, for illustration
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head

# step 1-- prepare a dataset (multivariate for demonstration)
X_train, y_train = load_arrow_head(split="train", return_type="numpy3D")
X_new, _ = load_arrow_head(split="test", return_type="numpy3D")

In [ ]:
X_train.shape, y_train.shape

In [ ]:
#X_train

In [ ]:
import sktime.classification.deep_learning as dl_clf
from sktime.classification.deep_learning import GRUFCNNClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_type="numpy3D")
X_test, y_test = load_unit_test(split="test", return_type="numpy3D")

In [ ]:
X_train.shape

In [ ]:
cnn = GRUFCNNClassifier( hidden_dim=128, gru_layers=2, verbose=True)
cnn.fit(X_train, y_train)